##  Definire e identificare le caratteristiche significative

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Caricamento dei dati
file_path = 'csv/data_pre_processing.csv'  # Sostituisci con il percorso corretto del tuo file CSV
data = pd.read_csv(file_path, low_memory=False)

# Convertiamo le colonne 'value' e 'value_label' in float, forzando i valori non numerici a NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Pulizia dei dati
data_cleaned = data.drop(columns=['measure_id', 'oid', 'measure_name', 'device_name', 'ip_address'])

# Gestione dei valori mancanti, se presenti
data_cleaned = data_cleaned.dropna()

# Conversione delle colonne categoriali in stringhe
categorical_columns = ['event_operator', 'event_variable']
for col in categorical_columns:
    data_cleaned[col] = data_cleaned[col].astype(str)

# Codifica delle variabili categoriali
label_encoder = LabelEncoder()
for col in categorical_columns:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])

# Creazione delle etichette
data_cleaned['anomaly'] = data['event_operator'].apply(lambda x: 1 if x == 'higher' else 0)

# Selezione delle caratteristiche (features) e etichetta (label)
X = data_cleaned.drop(columns=['anomaly', 'measure_date'])
y = data_cleaned['anomaly']

# Divisione del dataset in train e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Salvataggio dei dati pre-elaborati (opzionale)
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)


## Bilanciamento del dataset

In [2]:
from imblearn.over_sampling import SMOTE

# Bilanciamento del dataset usando SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


## Addestramento del modello Isolation Forest
Più appropriato per rilevare anomalie, soprattutto quando si lavora con dati sbilanciati o quando l'obiettivo principale è identificare eventi rari o anomali.

Addestriamo un modello IsolationForest per il rilevamento delle anomalie e successivamente un modello RandomForestClassifier per classificare ulteriormente i dati sulla base delle etichette generate dalla IsolationForest.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# Caricamento dei dati
file_path = 'csv/data_pre_processing.csv'  # Sostituisci con il percorso corretto del tuo file CSV
data = pd.read_csv(file_path, low_memory=False)

# Convertiamo le colonne 'value' e 'value_label' in float, forzando i valori non numerici a NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Pulizia dei dati
data_cleaned = data.drop(columns=['measure_id', 'oid', 'measure_name', 'device_name', 'ip_address'])

# Gestione dei valori mancanti, se presenti
data_cleaned = data_cleaned.dropna()

# Conversione delle colonne categoriali in stringhe
categorical_columns = ['event_operator', 'event_variable']
for col in categorical_columns:
    data_cleaned[col] = data_cleaned[col].astype(str)

# Codifica delle variabili categoriali
label_encoder = LabelEncoder()
for col in categorical_columns:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])

# Selezione delle caratteristiche (features)
X = data_cleaned.drop(columns=['measure_date'])

# Addestramento del modello Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X)

# Predizione sul set di dati
data_cleaned['scores'] = iso_forest.decision_function(X)
data_cleaned['anomaly'] = iso_forest.predict(X)

# L'etichetta -1 indica un'anomalia e 1 indica normale
data_cleaned['anomaly'] = data_cleaned['anomaly'].map({1: 0, -1: 1})

# Divisione del dataset in train e test per valutare il modello
X_train, X_test, y_train, y_test = train_test_split(X, data_cleaned['anomaly'], test_size=0.2, random_state=42)

# Addestramento del modello Random Forest sul dataset bilanciato
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predizione sul set di test
y_pred = clf.predict(X_test)

# Valutazione del modello
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Salvataggio dei modelli addestrati (opzionale)
joblib.dump(iso_forest, 'isolation_forest_model.joblib')
joblib.dump(clf, 'random_forest_classifier_model.joblib')


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1534065
           1       1.00      1.00      1.00     80704

    accuracy                           1.00   1614769
   macro avg       1.00      1.00      1.00   1614769
weighted avg       1.00      1.00      1.00   1614769

Confusion Matrix:
 [[1534065       0]
 [      0   80704]]


['random_forest_classifier_model.joblib']

# cross-validation più rigorosa e l'aggiunta di rumore ai dati

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import numpy as np

# Caricamento dei dati
file_path = 'csv/data_pre_processing.csv'  # Sostituisci con il percorso corretto del tuo file CSV
data = pd.read_csv(file_path, low_memory=False)

# Convertiamo le colonne 'value' e 'value_label' in float, forzando i valori non numerici a NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Pulizia dei dati
data_cleaned = data.drop(columns=['measure_id', 'oid', 'measure_name', 'device_name', 'ip_address'])

# Gestione dei valori mancanti, se presenti
data_cleaned = data_cleaned.dropna()

# Conversione delle colonne categoriali in stringhe
categorical_columns = ['event_operator', 'event_variable']
for col in categorical_columns:
    data_cleaned[col] = data_cleaned[col].astype(str)

# Codifica delle variabili categoriali
label_encoder = LabelEncoder()
for col in categorical_columns:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])

# Selezione delle caratteristiche (features)
X = data_cleaned.drop(columns=['measure_date'])

# Addestramento del modello Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X)

# Predizione sul set di dati
data_cleaned['scores'] = iso_forest.decision_function(X)
data_cleaned['anomaly'] = iso_forest.predict(X)

# L'etichetta -1 indica un'anomalia e 1 indica normale
data_cleaned['anomaly'] = data_cleaned['anomaly'].map({1: 0, -1: 1})

# Divisione del dataset in train e test per valutare il modello
X_train, X_test, y_train, y_test = train_test_split(X, data_cleaned['anomaly'], test_size=0.2, random_state=42)

# Addestramento del modello Random Forest sul dataset bilanciato
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Cross-validation
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

# Predizione sul set di test
y_pred = clf.predict(X_test)

# Valutazione del modello
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Aggiunta di rumore ai dati di test
X_test_noisy = X_test + np.random.normal(0, 0.1, X_test.shape)

# Predizione sui dati di test con rumore
y_pred_noisy = clf.predict(X_test_noisy)

# Valutazione del modello con dati rumorosi
print("Accuracy with noisy data:", accuracy_score(y_test, y_pred_noisy))
print("Classification Report with noisy data:\n", classification_report(y_test, y_pred_noisy))
print("Confusion Matrix with noisy data:\n", confusion_matrix(y_test, y_pred_noisy))

# Salvataggio dei modelli addestrati (opzionale)
joblib.dump(iso_forest, 'isolation_forest_model.joblib')
joblib.dump(clf, 'random_forest_classifier_model.joblib')


Cross-validation scores: [1. 1. 1. 1. 1.]
Mean cross-validation score: 1.0
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1534065
           1       1.00      1.00      1.00     80704

    accuracy                           1.00   1614769
   macro avg       1.00      1.00      1.00   1614769
weighted avg       1.00      1.00      1.00   1614769

Confusion Matrix:
 [[1534065       0]
 [      0   80704]]
Accuracy with noisy data: 0.9999269245322396
Classification Report with noisy data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1534065
           1       1.00      1.00      1.00     80704

    accuracy                           1.00   1614769
   macro avg       1.00      1.00      1.00   1614769
weighted avg       1.00      1.00      1.00   1614769

Confusion Matrix with noisy data:
 [[1534011      54]
 [     64   80640]]


['random_forest_classifier_model.joblib']

## Esplorazione e com prensione dei dati

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Caricamento dei dati
file_path = 'csv/data_pre_processing.csv'  # Sostituisci con il percorso corretto del tuo file CSV
data = pd.read_csv(file_path, low_memory=False)

# Convertiamo le colonne 'value' e 'value_label' in float, forzando i valori non numerici a NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Pulizia dei dati
data_cleaned = data.drop(columns=['measure_id', 'oid', 'measure_name', 'device_name', 'ip_address'])

# Gestione dei valori mancanti, se presenti
data_cleaned = data_cleaned.dropna()

# Conversione delle colonne categoriali in stringhe
categorical_columns = ['event_operator', 'event_variable']
for col in categorical_columns:
    data_cleaned[col] = data_cleaned[col].astype(str)

# Codifica delle variabili categoriali
label_encoder = LabelEncoder()
for col in categorical_columns:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])

# Selezione delle caratteristiche (features)
X = data_cleaned.drop(columns=['measure_date'])

# Addestramento del modello Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X)

# Predizione sul set di dati
data_cleaned['scores'] = iso_forest.decision_function(X)
data_cleaned['anomaly'] = iso_forest.predict(X)

# L'etichetta -1 indica un'anomalia e 1 indica normale
data_cleaned['anomaly'] = data_cleaned['anomaly'].map({1: 0, -1: 1})

# Visualizzazione della distribuzione delle classi
sns.countplot(data_cleaned['anomaly'])
plt.title('Distribuzione delle classi')
plt.show()

# Visualizzazione delle correlazioni tra le caratteristiche
plt.figure(figsize=(12, 8))
sns.heatmap(data_cleaned.corr(), annot=True, fmt=".2f")
plt.title('Matrice di correlazione')
plt.show()

# Divisione del dataset in train e test per valutare il modello
X_train, X_test, y_train, y_test = train_test_split(X, data_cleaned['anomaly'], test_size=0.2, random_state=42)

# Addestramento del modello Random Forest sul dataset bilanciato
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Cross-validation
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

# Predizione sul set di test
y_pred = clf.predict(X_test)

# Valutazione del modello
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Aggiunta di rumore ai dati di test
X_test_noisy = X_test + np.random.normal(0, 0.1, X_test.shape)

# Predizione sui dati di test con rumore
y_pred_noisy = clf.predict(X_test_noisy)

# Valutazione del modello con dati rumorosi
print("Accuracy with noisy data:", accuracy_score(y_test, y_pred_noisy))
print("Classification Report with noisy data:\n", classification_report(y_test, y_pred_noisy))
print("Confusion Matrix with noisy data:\n", confusion_matrix(y_test, y_pred_noisy))

# Salvataggio dei modelli addestrati (opzionale)
joblib.dump(iso_forest, 'isolation_forest_model.joblib')
joblib.dump(clf, 'random_forest_classifier_model.joblib')


: 

# Rimozione di alcune caratteristiche

In [ ]:
# Rimuovi alcune caratteristiche casuali
X_reduced = X.drop(columns=['oid', 'ip_address', 'event_variable'])  

# Divisione del dataset in train e test per valutare il modello con caratteristiche ridotte
X_train, X_test, y_train, y_test = train_test_split(X_reduced, data_cleaned['anomaly'], test_size=0.2, random_state=42)

# Addestramento del modello Random Forest sul dataset ridotto
clf_reduced = RandomForestClassifier(n_estimators=100, random_state=42)
clf_reduced.fit(X_train, y_train)

# Predizione sul set di test
y_pred_reduced = clf_reduced.predict(X_test)

# Valutazione del modello
print("Accuracy with reduced features:", accuracy_score(y_test, y_pred_reduced))
print("Classification Report with reduced features:\n", classification_report(y_test, y_pred_reduced))
print("Confusion Matrix with reduced features:\n", confusion_matrix(y_test, y_pred_reduced))


## Utilizzo di tecniche di ML più avanzate
### Utilizziamo XGBoost
XGBoost comnprende un algoritmo di boosting, metodo di ensemble che combina più modelli deboli (come gli alberi di decisione) per creare un modello più forte

In [ ]:
import xgboost as xgb

# Addestramento del modello XGBoost
xgb_clf = xgb.XGBClassifier(random_state=42)
xgb_clf.fit(X_train, y_train)

# Predizione sul set di test
y_pred_xgb = xgb_clf.predict(X_test)

# Valutazione del modello
print("Accuracy with XGBoost:", accuracy_score(y_test, y_pred_xgb))
print("Classification Report with XGBoost:\n", classification_report(y_test, y_pred_xgb))
print("Confusion Matrix with XGBoost:\n", confusion_matrix(y_test, y_pred_xgb))


In [ ]:
from sklearn.ensemble import VotingClassifier

# Creazione di un ensemble di modelli
voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('xgb', xgb.XGBClassifier(random_state=42)),
        ('iso', iso_forest)  # Anche se IsolationForest non è un classificatore diretto, puoi combinare più modelli se preferisci
    ],
    voting='soft'  # Usa 'hard' per il metodo di maggioranza
)

# Addestramento del modello ensemble
voting_clf.fit(X_train, y_train)

# Predizione sul set di test
y_pred_voting = voting_clf.predict(X_test)

# Valutazione del modello
print("Accuracy with Voting Classifier:", accuracy_score(y_test, y_pred_voting))
print("Classification Report with Voting Classifier:\n", classification_report(y_test, y_pred_voting))
print("Confusion Matrix with Voting Classifier:\n", confusion_matrix(y_test, y_pred_voting))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Caricamento dei dati
file_path = 'csv/data_pre_processing.csv'  # Sostituisci con il percorso corretto del tuo file CSV
data = pd.read_csv(file_path, low_memory=False)

# Convertiamo le colonne 'value' e 'value_label' in float, forzando i valori non numerici a NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Pulizia dei dati
data_cleaned = data.drop(columns=['measure_id', 'oid', 'measure_name', 'device_name', 'ip_address'])

# Gestione dei valori mancanti, se presenti
data_cleaned = data_cleaned.dropna()

# Conversione delle colonne categoriali in stringhe
categorical_columns = ['event_operator', 'event_variable']
for col in categorical_columns:
    data_cleaned[col] = data_cleaned[col].astype(str)

# Codifica delle variabili categoriali
label_encoder = LabelEncoder()
for col in categorical_columns:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])

# Selezione delle caratteristiche (features)
X = data_cleaned.drop(columns=['measure_date'])

# Addestramento del modello Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X)

# Predizione sul set di dati
data_cleaned['scores'] = iso_forest.decision_function(X)
data_cleaned['anomaly'] = iso_forest.predict(X)

# L'etichetta -1 indica un'anomalia e 1 indica normale
data_cleaned['anomaly'] = data_cleaned['anomaly'].map({1: 0, -1: 1})

# Divisione del dataset in train e test per valutare il modello
X_train, X_test, y_train, y_test = train_test_split(X, data_cleaned['anomaly'], test_size=0.2, random_state=42)

# Predizione sul set di test
y_pred = iso_forest.predict(X_test)

# Mappatura delle predizioni per la valutazione
y_pred = pd.Series(y_pred).map({1: 0, -1: 1})

# Valutazione del modello
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Salvataggio del modello addestrato (opzionale)
import joblib
joblib.dump(iso_forest, 'isolation_forest_model2.joblib')
